In [6]:
import pickle
import numpy as np
import pandas as pd
import math
from NLTK_cleaning import clean 
# Fuction to determine tf of word in query with respect to query as text
def tf(term, text):
    term_count = text.count(term)
    total_words = len(text)
    return term_count / total_words  
# Fuction to determine tf of word in query with respect to database as
def idf(term, df):
    count = df["cleaned description"].apply(lambda x: term in x if isinstance(x, str) else False).sum()
    return math.log((len(df)+1)/(count)+1) +1    
# Function to determine cosine similarity between two vectors
def calculate_cosine_similarity(query_tfidf, document_tfidf):
    dot_product = np.dot(np.array(query_tfidf), np.array(document_tfidf))
    norm_query = np.linalg.norm(np.array(query_tfidf))
    norm_document = np.linalg.norm(np.array(document_tfidf))
    if norm_query == 0 or norm_document == 0:
        return 0.0 
    cosine_similarity = dot_product / (norm_query * norm_document)
    return cosine_similarity
# Function to search corpus for matches in query and return matched sorted by tfidf vector similarity
def search(query):
    
    # we clean our query using the nltk cleaning function defined in nltk_cleaning.py
    query = clean(query).lower().split(" ")
    
    # collect all matches for each word across documents and sore them as a union of matches of document
    match_list = []    
    for lemma in query:
        word_id = vocab[vocab['term'] == lemma]['term_id']
        try:
            word_id = word_id.values[0]
            matches = set(inverted_index[word_id]) 
            match_list.append(matches)
        except:
            print(f"{lemma} not in dataset. It will not be included in search.")
            query.remove(lemma)

    match_list = set.union(*match_list)   

    # We calculate the tfidf of the query
    query_tfidf={}
    tfidf_query_corpus = {}
    for word in query:
        tf_query = tf(word,(query))
        idf_query = idf(word, degrees)
        query_tfidf[word] = tf_query*idf_query

    # we retreive from our tfidf inverted index dictionary the tfidf of each word in the query for each document in our match_list
    # and create a dictionary of document index as key, and cosing similarity as value
    page_rank ={}
    for result in match_list:
        doc = []
        for lemma in query:
            word_id = vocab[vocab['term'] == lemma]['term_id'].values[0] #get word ID from vocab list
            try:
                doc.append([x[1] for x in tfidf_dict[word_id] if x[0]== result][0])
            except:
                doc.append(0)   # if word not in doc tfidf =0 
        tfidf_query_corpus[result] = doc
        page_rank[result] = calculate_cosine_similarity(list(query_tfidf.values()), tfidf_query_corpus[result])
    # we sort our dictionary and return the top 10 results
    page_rank = dict(sorted(page_rank.items(), key=lambda item: item[1], reverse=True)) 
    page_rank_index = list(page_rank.keys())[0:10]
    return page_rank_index

with open(r"C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\saved_dictionary_tfidf.pkl", 'rb') as f:
    tfidf_dict = pickle.load(f)
with open(r"C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\saved_dictionary.pkl", 'rb') as f:
    inverted_index = pickle.load(f)

vocab_path = r'C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\Vocabulary.csv'
vocab = pd.read_csv(vocab_path)

degrees = pd.read_csv(r"C:\Users\youse\Desktop\GitHub\adm-hw3\Databases\Parsed_database.csv")
degrees = degrees.drop(columns=["Unnamed: 0"])





### Write your search query under the variable query and run.

In [10]:
query = "masters in data science, specificially big data"

# here we create a database to display our data
page_rank_index = search(query)
all_results = pd.DataFrame(columns=["courseName", "universityName", "description", "url"])
if len(page_rank_index) > 0 :
    for res in page_rank_index:
        course = degrees["courseName"][res]
        uni = degrees["universityName"][res]
        desc = " ".join(degrees["description"][res].split())
        url = degrees["url"][res]
        all_results.loc[len(all_results)] = [course, uni, desc, url]
        
else:
    print("No result was found meeting all criteria.")    
all_results.head(10) 

specificially not in dataset. It will not be included in search.


,courseName,universityName,description,url
0,Applied Sciences and Engineering: Applied Comp...,Vrije Universiteit Brussel,Data is the future. Our world depends on it. C...,https://www.findamasters.com/masters-degrees/c...
1,MSc Big Data Analytics,University of Liverpool,Further your computer science career with a sp...,https://www.findamasters.com/masters-degrees/c...
2,Big Data MSc,University of Stirling,Big data is increasingly important in today’s ...,https://www.findamasters.com/masters-degrees/c...
3,Data Analytics (MSc),George Washington University,Administered jointly through the Department of...,https://www.findamasters.com/masters-degrees/c...
4,Data Science - MSc,University of Glasgow,The Masters in Data Science is a specialist ve...,https://www.findamasters.com/masters-degrees/c...
5,Data Science - MSc/PgD/PgC,Cardiff Metropolitan University,This Master's degree in Data Science is an ind...,https://www.findamasters.com/masters-degrees/c...
6,Data Science for Politics & Policymaking MSc,University of Strathclyde,The MSc in Data Science for Politics & Policym...,https://www.findamasters.com/masters-degrees/c...
7,ICT for Internet and Multimedia,University of Padua,The Master’s degree in ICT for Internet and Mu...,https://www.findamasters.com/masters-degrees/c...
8,Master of Science in Big Data Analytics,Atlantic Technological University,Our Master of Science in Computing in Big Data...,https://www.findamasters.com/masters-degrees/c...
9,Master of Science in Big Data Analytics and Ar...,Atlantic Technological University,Our Master of Science in Computing in Big Data...,https://www.findamasters.com/masters-degrees/c...
